In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargar el archivo de datos (ajusta la ruta a tu archivo)
df_business = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\locales_google.parquet')  # Tabla de negocios
df_reviews = pd.read_parquet(r'C:\Users\GASTON\Desktop\PROYECTO FINAL\DATA\ml_unificado.parquet')  # Tabla de reseñas con datos en 'text' unificada con los negocios


In [2]:
def weighted_rating(x, m, C):
    v = x['num_of_reviews']
    R = x['avg_rating']
    return (v / (v + m) * R) + (m / (v + m) * C)

# Recomendar locales por comida y ciudad

In [3]:
def recomendacion_comida_ciudad(df, tipo_comida, ciudad, top_n=5):
    '''
    Esta función recibe un DataFrame sobre el que trabaja, el tipo de comida y la ciudad que se busca.
    Devuelve una recomendación de los locales con mejor puntuación ponderada que coincidan con esos criterios.
    '''
    # Filtrar por tipo de comida y ciudad
    df_filtrado = df[(df['category'].str.contains(tipo_comida, case=False, na=False)) & 
                     (df['city'].str.contains(ciudad, case=False, na=False))]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes de {tipo_comida} en {ciudad}."
    
    # Agrupar por gmap_id para evitar duplicados
    df_agrupado = df_filtrado.groupby('gmap_id').agg({
        'name': 'first',  # Mantener el primer nombre del restaurante
        'address': 'first',  # Mantener la primera dirección del restaurante
        'category': 'first',  # Mantener la primera categoría
        'avg_rating': 'mean',  # Calcular el promedio de las calificaciones
        'num_of_reviews': 'sum',  # Sumar todas las reseñas
        'rating': 'mean'  # Calcular el promedio de las calificaciones de las reseñas
    }).reset_index()

    # Calcular la puntuación ponderada
    m = df_agrupado['num_of_reviews'].quantile(0.90)
    C = df_agrupado['avg_rating'].mean()
    df_agrupado['puntuacion'] = weighted_rating(df_agrupado, m, C)
    
    # Ordenar por puntuación
    df_top5 = df_agrupado.sort_values('puntuacion', ascending=False).head(top_n)
    
    # Seleccionar las columnas relevantes para mostrar
    return df_top5[['name', 'address', 'category', 'avg_rating', 'num_of_reviews', 'puntuacion']]

Pruebas

In [6]:
# Probar la función con el DataFrame unificado
recomendaciones_sabor = recomendacion_comida_ciudad(df_reviews, 'Burger', 'Los Angeles')
recomendaciones_sabor

,name,address,category,avg_rating,num_of_reviews,puntuacion
17,Bobo's Burgers,"Bobo's Burgers, 1220 Firestone Blvd, Los Angel...","Hamburger restaurant, Breakfast restaurant, Br...",4.4,198406,4.282660
9,Shake Shack,"Shake Shack, 6201 Hollywood Blvd, Los Angeles,...","Hamburger restaurant, American restaurant, Fas...",4.3,597510,4.268939
13,Cypress Best Burgers,"Cypress Best Burgers, 3240 N Figueroa St, Los ...",Hamburger restaurant,4.3,103886,4.234712
11,VOWburger,"VOWburger, 519 N Fairfax Ave, Los Angeles, CA ...","Vegan restaurant, Fast food restaurant, Hambur...",4.7,1534,4.217153
7,"The Stand ""American Classics Redefined""","The Stand ""American Classics Redefined"", 2000 ...","Restaurant, American restaurant, Caterer, Hamb...",4.4,1209,4.215648


In [22]:
# Probar la función con el DataFrame unificado
recomendaciones_sabor = recomendacion_comida_ciudad(df_reviews, 'Pizza', 'Pasadena')
recomendaciones_sabor

,name,address,category,avg_rating,num_of_reviews,puntuacion
1,U Street Pizza,"U Street Pizza, 33 Union St, Pasadena, CA 91103",Pizza restaurant,4.6,48,4.222514
2,Domino's Pizza,"Domino's Pizza, 1017 Fair Oaks Ave, South Pasa...","Pizza delivery, Delivery Restaurant, Takeout R...",4.6,18,4.208751
0,Pizza Man,"Pizza Man, 1306 E Colorado Blvd, Pasadena, CA ...","Pizza delivery, Takeout Restaurant",3.4,994,3.757928


# Recomendar locales por zona o ciudad

In [7]:
def recomendacion_por_zona(df, ciudad, top_n=5, min_reviews=10):
    '''
    Esta función recibe un DataFrame, y una ciudad o zona del Estado de California.
    Devuelve un top 10 de los mejores locales según la calificación ponderada.
    Pueden no haber reseñas suficientes en la ciudad o zona seleccionada.
    '''
    # Filtrar por ciudad
    df_filtrado = df[df['city'].str.contains(ciudad, case=False, na=False)]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes en {ciudad}."
    
    # Filtrar solo restaurantes con un mínimo de reseñas
    df_filtrado = df_filtrado[df_filtrado['num_of_reviews'] >= min_reviews]
    
    if df_filtrado.empty:
        return f"No se encontraron restaurantes con al menos {min_reviews} reseñas en {ciudad}."
    
    # Ordenar por calificación promedio y número de reseñas
    df_top5 = df_filtrado.sort_values(['avg_rating', 'num_of_reviews'], ascending=[False, False]).head(top_n)
    
    # Seleccionar las columnas relevantes para mostrar
    return df_top5[['name', 'avg_rating', 'num_of_reviews', 'category', 'address']]

Pruebas

In [24]:
# Probar la función con el DataFrame df_business
recomendaciones_zona = recomendacion_por_zona(df_business, 'Los Angeles', top_n=5, min_reviews=20)
recomendaciones_zona

,name,avg_rating,num_of_reviews,category,address
2504391,Juanito's Tacos,4.9,78,Mexican restaurant,"Juanito's Tacos, 1950 E Washington Blvd, Los A..."
1422509,Eszett,4.9,57,Restaurant,"Eszett, 3510 Sunset Blvd, Los Angeles, CA 90026"
1460247,Cafe De Mama,4.9,38,Coffee shop,"Cafe De Mama, 1102 S Western Ave, Los Angeles,..."
1817065,"50/50 Juice Lounge, Where Health Meets Juice 4...",4.9,38,Restaurant,"50/50 Juice Lounge, Where Health Meets Juice 4..."
2588766,Mo's House Of Axe * Temporarily Closed *,4.9,38,Restaurant,"Mo's House Of Axe * Temporarily Closed *, 611 ..."


# Recomendación de locales según palabras clave en las reseñas

In [10]:
def recomendacion_segun_palabras(df_unificado, palabras_clave, min_reviews=10):
    '''
    Esta función recibe un DataFrame y una lista de palabras clave.
    Busca las reseñas que tengan esas palabras clave, y las ordena según la puntuación ponderada.
    Devuelve una tabla con los locales mejor ponderados que respeten los criterios de búsqueda.
    '''
    # Unir las palabras clave en una expresión regular
    palabras_regex = '|'.join([f'\\b{palabra}\\b' for palabra in palabras_clave])
    
    # Filtrar las reseñas que contienen las palabras clave
    df_filtrado = df_unificado[df_unificado['text'].str.contains(palabras_regex, case=False, na=False)]
    
    if df_filtrado.empty:
        return "No se encontraron reseñas con las palabras clave."
    
    # Agrupar por gmap_id para evitar duplicados
    df_agrupado = df_filtrado.groupby('gmap_id').agg({
        'name': 'first',  # Mantener el primer nombre del restaurante
        'address': 'first',  # Mantener la primera dirección del restaurante
        'city': 'first',  # Mantener la primera ciudad
        'category': 'first',  # Mantener la primera categoría
        'avg_rating': 'mean',  # Promedio de la calificación
        'num_of_reviews': 'sum',  # Sumar el número total de reseñas
        'rating': 'mean',  # Promedio de las calificaciones de las reseñas
        'text': 'first'  # Mantener el primer texto de reseña
    }).reset_index()
    
    # Filtrar solo restaurantes con un mínimo de reseñas
    df_agrupado = df_agrupado[df_agrupado['num_of_reviews'] >= min_reviews]
    
    if df_agrupado.empty:
        return f"No se encontraron restaurantes con al menos {min_reviews} reseñas que contengan las palabras clave."
    
    # Ordenar por calificación promedio y número de reseñas
    df_top = df_agrupado.sort_values(['avg_rating', 'num_of_reviews'], ascending=[False, False])
    
    # Seleccionar las columnas relevantes para mostrar
    return df_top[['name', 'address', 'city', 'category', 'avg_rating', 'num_of_reviews', 'text']].head(10)

Pruebas

In [11]:
# Probar la función con df_unificado
palabras_clave = ['amazing', 'good burger', 'very good', 'friendly']
resultados_busqueda = recomendacion_segun_palabras(df_reviews, palabras_clave)
resultados_busqueda

,name,address,city,category,avg_rating,num_of_reviews,text
212,LASO /Nepalese Fusion,"LASO /Nepalese Fusion, 101 Antonina Ave Suite ...",American Canyon,Fusion restaurant,5.0,225,Wife and I had an excellent dinner at the outd...
3881,The Kilted Cake and Supply,"The Kilted Cake and Supply, 28061 Jefferson Av...",Temecula,"Baking supply store, Airbrushing supply store,...",5.0,192,So nice! Extremely organized. Amazing customer...
4175,In-N-Out Burger,"In-N-Out Burger, 3483 Van Buren Boulevard, Riv...",Riverside,"Hamburger restaurant, American restaurant, Din...",5.0,146,Very good service dine in or dine out its a gr...
1996,North Fork Chai Co.,"North Fork Chai Co., 661 Suite B, Newcastle Rd...",Newcastle,Cafe,5.0,92,Great place to come to for some tra or coffee....
4003,CALVIN'S KOREAN HOT CHICKEN,"CALVIN'S KOREAN HOT CHICKEN, 3211 Holiday Ct S...",La Jolla,"Fried chicken takeaway, Chicken shop, Chicken ...",5.0,92,Wow. So very good. Also they have gluten free ...
4921,Reverent Coffee Bar,"Reverent Coffee Bar, 627 Deep Valley Dr Suite ...",Rolling Hills Estates,Coffee shop,5.0,78,I've been here only a few times so far - which...
137,Quick Sushi Santa Rosa,"Quick Sushi Santa Rosa, 2500 Mendocino Ave, Sa...",Santa Rosa,Japanese restaurant,5.0,74,Very good! Bummer they don't open on Sundays. ...
525,Ponsford's Place Bakery and Innovation Center....,Ponsford's Place Bakery and Innovation Center....,San Rafael,Bakery,5.0,50,Absolutely beautiful and delicious pastries! I...
3817,Gabino's Creperie,"Gabino's Creperie, 170 E Palm Canyon Dr #4, Pa...",Palm Springs,Restaurant,5.0,48,I would start with saying this place is a hidd...
4043,American Heroes & Brew,"American Heroes & Brew, 300 Carlsbad Village D...",Carlsbad,Restaurant,5.0,48,"This place is the best! Amazing food, great at..."


In [25]:
# Probar la función con df_unificado
palabras_clave = ['great burger']
resultados_busqueda = recomendacion_segun_palabras(df_reviews, palabras_clave)
resultados_busqueda

,name,address,city,category,avg_rating,num_of_reviews,text
26,Red Moose Cafe & Inn,"Red Moose Cafe & Inn, 224 Main St, Sierra City...",Sierra City,Restaurant,4.8,37,We had great burger and fish and chips.
20,In-N-Out Burger,"In-N-Out Burger, 8010 N Blackstone Ave, Fresno...",Fresno,"Hamburger restaurant, American restaurant, Din...",4.7,12040,Very great burger and very clean 👍
40,Bun Street,"Bun Street, 180 S Glendora Ave, West Covina, C...",West Covina,Hamburger restaurant,4.7,828,Great place to sit and grab a great burger.
33,VOWburger,"VOWburger, 519 N Fairfax Ave, Los Angeles, CA ...",Los Angeles,"Vegan restaurant, Fast food restaurant, Hambur...",4.7,118,Great burger..just like In-n-Out but vegan! Gr...
1,In-N-Out Burger,"In-N-Out Burger, 1275 Dana Dr, Redding, CA 960...",Redding,"Hamburger restaurant, American restaurant, Din...",4.6,20272,Love the pure simple ingredients. They make a...
23,In-N-Out Burger,"In-N-Out Burger, 1933 S Mooney Blvd, Visalia, ...",Visalia,"Hamburger restaurant, American restaurant, Din...",4.6,16312,Always a great burger weather your in Visalia ...
51,In-N-Out Burger,"In-N-Out Burger, 4115 Campus Dr, Irvine, CA 92...",Irvine,"Hamburger restaurant, American restaurant, Din...",4.6,10990,Great burger at a great price. Staff is always...
60,Crazee Burger,"Crazee Burger, 3993 30th St, San Diego, CA 921...",San Diego,Hamburger restaurant,4.6,10906,Great Burger spot serving a range of options f...
63,In-N-Out Burger,"In-N-Out Burger, 600 S Brookhurst St, Anaheim,...",Anaheim,"Hamburger restaurant, American restaurant, Din...",4.6,10857,Great burger and great prices
27,In-N-Out Burger,"In-N-Out Burger, 820 W Imola Ave, Napa, CA 945...",Napa,"Hamburger restaurant, American restaurant, Din...",4.6,5226,This is the busiest place at lunch and worth t...


# Recomendación de locales en base a reseñas similares

In [14]:
def recomendacion_reviews_similares(df_unificado, nombre_restaurante, top_n=5):
    '''
    Esta función recibe un dataframe, y el nombre de un local gastronómico.
    Devolverá una recomendación de una categoría similar, y con reseñas similares.
    '''
    # Filtrar el DataFrame para encontrar el restaurante
    df_restaurante = df_unificado[df_unificado['name'].str.contains(nombre_restaurante, case=False, na=False)]
    
    if df_restaurante.empty:
        return f"No se encontraron restaurantes con el nombre {nombre_restaurante}."
    
    # Obtener el gmap_id del restaurante
    gmap_id_restaurante = df_restaurante.iloc[0]['gmap_id']
    
    # Filtrar las reseñas que tienen texto y corresponden a este restaurante
    df_reseñas_restaurante = df_unificado[(df_unificado['text'].notnull()) & (df_unificado['gmap_id'] == gmap_id_restaurante)]
    
    if df_reseñas_restaurante.empty:
        return f"No se encontraron reseñas para el restaurante {nombre_restaurante}."
    
    # Crear la matriz TF-IDF para las reseñas combinadas
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix_reseñas = tfidf.fit_transform(df_reseñas_restaurante['category_text'])
    tfidf_matrix_todas_reseñas = tfidf.transform(df_unificado['category_text'])
    
    # Calcular la similitud del coseno entre las reseñas del restaurante y todas las reseñas
    cosine_sim = cosine_similarity(tfidf_matrix_reseñas, tfidf_matrix_todas_reseñas)
    
    # Obtener el índice de reseñas más similares
    sim_scores = list(enumerate(cosine_sim.mean(axis=0)))  # Usar la media de similitud para comparar con todas las reseñas
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:top_n]
    
    # Obtener los índices de los restaurantes recomendados
    restaurant_indices = [i[0] for i in sim_scores]
    
    # Asegurarse de que los índices están dentro del rango del DataFrame
    if restaurant_indices:
        # Obtener información de los restaurantes recomendados
        recomendaciones = df_unificado.iloc[restaurant_indices].drop_duplicates(subset='gmap_id')
        recomendaciones = recomendaciones[['name', 'address', 'city', 'category', 'avg_rating', 'num_of_reviews', 'text']].reset_index(drop=True)
    else:
        recomendaciones = pd.DataFrame()  # Devolver un DataFrame vacío si no hay recomendaciones
    
    return recomendaciones

Pruebas

In [17]:
nombre_restaurante = '21 Biryani Pots'
resultados_recomendaciones = recomendacion_reviews_similares(df_reviews, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,text
0,Paradise Biryani Pointe,"Paradise Biryani Pointe, 8995 Mira Mesa Blvd S...",San Diego,Indian restaurant,3.5,618,Authentic biryani place. Their biryani is very...
1,Sakoon,"Sakoon, 357 Castro St, Mountain View, CA 94041...",Mountain View,"Indian restaurant, Bar",4.2,2474,Very Authentic Indian food.\nHappy Hour for dr...
2,Biryani House,"Biryani House, 949 Ruff Dr, San Jose, CA 95110",San Jose,"Indian takeaway, Biryani restaurant, Indian re...",4.7,68,Incredible place that deserves more ratings an...


In [20]:
nombre_restaurante = 'Vons Chicken'
resultados_recomendaciones = recomendacion_reviews_similares(df_reviews, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,text
0,Crab House at Pier 39,"Crab House at Pier 39, San Francisco, 203 Pier...",San Francisco,"Crab house, American restaurant, Bar, Californ...",4.2,2068,(Translated by Google) It has a different expe...
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",La Mirada,Restaurant,4.5,18,(Translated by Google) There is a different me...
2,Hong Kong Flower Lounge Restaurant 香滿樓,"Hong Kong Flower Lounge Restaurant 香滿樓, 51 E M...",Millbrae,"Chinese restaurant, Dim sum restaurant, Seafoo...",3.9,918,(Translated by Google) The taste is good.\n\nH...
3,Tom's Jr,"Tom's Jr, 1725 N Long Beach Blvd, Compton, CA ...",Compton,"Hamburger restaurant, Fast food restaurant, Re...",4.4,1248,(Translated by Google) I love buying the famil...
4,Olympian Burgers,"Olympian Burgers, 2701 S Vermont Ave, Los Ange...",Los Angeles,"Family restaurant, American restaurant, Fast f...",4.5,1500,(Translated by Google) A place with many years...


In [26]:
nombre_restaurante = 'Pizza Hut'
resultados_recomendaciones = recomendacion_reviews_similares(df_reviews, nombre_restaurante)
resultados_recomendaciones

,name,address,city,category,avg_rating,num_of_reviews,text
0,Pizza Hut,"Pizza Hut, 7750 El Camino Real suite e, Carlsb...",Carlsbad,"Pizza restaurant, Chicken wings restaurant, De...",3.6,28,Good
1,Pizza Hut,"Pizza Hut, 8947 W Pico Blvd, Los Angeles, CA 9...",Los Angeles,"Pizza restaurant, Chicken wings restaurant, De...",3.0,48,It's okay
2,Pizza Hut,"Pizza Hut, 415 E Avenida Pico Ste K, San Cleme...",San Clemente,"Pizza restaurant, Chicken wings restaurant, De...",2.9,38,I love this place...greatest ever
3,Pizza Hut,"Pizza Hut, 92 W Court St, Woodland, CA 95695",Woodland,"Pizza restaurant, Chicken wings restaurant, De...",3.7,38,I loved it😁
